<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Removing-Duplicates" data-toc-modified-id="Removing-Duplicates-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Removing Duplicates</a></span></li><li><span><a href="#Transforming-Data-Using-a-Function-or-Mapping" data-toc-modified-id="Transforming-Data-Using-a-Function-or-Mapping-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Transforming Data Using a Function or Mapping</a></span></li><li><span><a href="#Replacing-Values" data-toc-modified-id="Replacing-Values-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Replacing Values</a></span></li><li><span><a href="#Renaming-Axis-Indexes" data-toc-modified-id="Renaming-Axis-Indexes-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Renaming Axis Indexes</a></span></li><li><span><a href="#Discretization-and-Binning" data-toc-modified-id="Discretization-and-Binning-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Discretization and Binning</a></span></li><li><span><a href="#Detecting-and-Filtering-Outliers" data-toc-modified-id="Detecting-and-Filtering-Outliers-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Detecting and Filtering Outliers</a></span></li><li><span><a href="#Permutation-and-Random-Sampling" data-toc-modified-id="Permutation-and-Random-Sampling-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Permutation and Random Sampling</a></span></li><li><span><a href="#Computing-Indicator/Dummy-Variables" data-toc-modified-id="Computing-Indicator/Dummy-Variables-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Computing Indicator/Dummy Variables</a></span></li></ul></div>

# 7.2 Data Transformation
### Removing Duplicates
Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method `duplicated` returns a boolean Series indicating whether each row is a duplicate (has been observed in a previous row) or not:

In [3]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

**Relatedly, `drop_duplicates` returns a DataFrame where the duplicated array is False:**

In [4]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


* Both of these methods by **default consider all of the columns**; alternatively, you can specify any subset of them to detect duplicates.
* Suppose we had an additional column of values and wanted to filter duplicates only based on the 'k1' column:

In [5]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [6]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


* `duplicated` and `drop_duplicates` by default keep the first observed value combination.
* Passing `keep='last'` will return the last one:

In [7]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping
For many datasets, you may wish to perform some transformation based on the values in an array, Series, or column in a DataFrame. Consider the following hypothetical data collected about various kinds of meat:

In [8]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon','Pastrami', 'corned beef', 'Bacon',\
                              'pastrami', 'honey ham', 'nova lox'],'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Suppose you wanted to add a column indicating the type of animal that each food came from.    
Let’s write down a mapping of each distinct meat type to the kind of animal:

In [9]:
meat_to_animal = {
'bacon': 'pig',
'pulled pork': 'pig',
'pastrami': 'cow',
'corned beef': 'cow',
'honey ham': 'pig',
'nova lox': 'salmon'
}

The map method on a Series accepts a function or dict-like object containing a mapping
* but here we have a small problem in that some of the meats are capitalized and others are not. 
* Thus, we need to convert each value to lowercase using the `str.lower` Series method:

In [10]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [11]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


**We could also have passed a function that does all the work:**

In [12]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

**Using map is a convenient way to perform element-wise transformations and other data cleaning–related operations.**

### Replacing Values
Filling in missing data with the `fillna` method is a special case of more general value replacement.
* As you’ve already seen, map can be used to modify a subset of values in an object but replace provides a simpler and more flexible way to do so. Let’s consider this Series:

In [13]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The -999 values might be sentinel values for missing data. 
To replace these with NA values that pandas understands, we can use `replace`, producing a new Series (unless you pass `inplace=True`):

In [14]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

**If you want to replace multiple values at once, you instead pass a list and then the substitute value:**

In [15]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

**To use a different replacement for each value, pass a list of substitutes:**

In [16]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

**The argument passed can also be a dict:**

In [17]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The `data.replace` method is distinct from `data.str.replace`, which performs string substitution element-wise. 

### Renaming Axis Indexes
* Like values in a Series, axis labels can be similarly transformed by a function or mapping of some form to produce new, differently labeled objects.

In [18]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),index=['Ohio', 'Colorado', 'New York'],\
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [19]:
transform = lambda x: x[:].upper()
data.index.map(transform)

Index(['OHIO', 'COLORADO', 'NEW YORK'], dtype='object')

You can assign to `index`, modifying the DataFrame in-place:

In [20]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


If you want to create a transformed version of a dataset **without modifying the original**, a useful method is `rename`:

In [21]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


Notably, rename can be used in conjunction with a dict-like object providing new values for a subset of the axis labels:

In [22]:
data.rename(index={'OHIO': 'INDIANA'},columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


### Discretization and Binning
Continuous data is often discretized or otherwise separated into “bins” for analysis

In [23]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. 

In [24]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special `Categorical` object.
* You can treat it like an array of strings indicating the bin name
* internally it contains a categories array specifying the distinct category names along with a labeling for the `ages` data in the `codes` attribute

In [25]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [26]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [27]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

You can change which side is closed by passing `right=False`:

In [28]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

You can also pass your own bin names by passing a list or array to the `labels` option:

In [29]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [30]:
pd.value_counts(pd.cut(ages, bins, labels=group_names))

Youth         5
MiddleAged    3
YoungAdult    3
Senior        1
dtype: int64

If you **pass an integer number of bins to cut instead of explicit bin edges**, it will compute equal-length bins based on the minimum and maximum values in the data

In [31]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.74, 0.98], (0.018, 0.26], (0.26, 0.5], (0.018, 0.26], (0.26, 0.5], ..., (0.74, 0.98], (0.5, 0.74], (0.018, 0.26], (0.26, 0.5], (0.018, 0.26]]
Length: 20
Categories (4, interval[float64]): [(0.018, 0.26] < (0.26, 0.5] < (0.5, 0.74] < (0.74, 0.98]]

The `precision=2` option limits the decimal precision to two digits.

A closely related function, `qcut`, bins the data **based on sample quantiles.**

In [32]:
data = np.random.randn(1000) # Normally distributed
cats = pd.qcut(data, 4) # Cut into quartiles
cats

[(-0.0148, 0.668], (-3.282, -0.721], (-3.282, -0.721], (-3.282, -0.721], (-3.282, -0.721], ..., (-0.721, -0.0148], (-0.0148, 0.668], (0.668, 3.169], (-0.0148, 0.668], (-0.0148, 0.668]]
Length: 1000
Categories (4, interval[float64]): [(-3.282, -0.721] < (-0.721, -0.0148] < (-0.0148, 0.668] < (0.668, 3.169]]

In [33]:
pd.value_counts(cats)

(0.668, 3.169]       250
(-0.0148, 0.668]     250
(-0.721, -0.0148]    250
(-3.282, -0.721]     250
dtype: int64

Similar to cut **you can pass your own quantiles**(numbers between 0 and 1, inclusive)

In [34]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-0.0148, 1.206], (-1.298, -0.0148], (-1.298, -0.0148], (-1.298, -0.0148], (-3.282, -1.298], ..., (-1.298, -0.0148], (-0.0148, 1.206], (-0.0148, 1.206], (-0.0148, 1.206], (-0.0148, 1.206]]
Length: 1000
Categories (4, interval[float64]): [(-3.282, -1.298] < (-1.298, -0.0148] < (-0.0148, 1.206] < (1.206, 3.169]]

### Detecting and Filtering Outliers
Filtering or transforming outliers is largely a matter of applying array operations

In [35]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.042987,-0.036197,0.002366,-0.009006
std,0.935083,1.035791,0.989870,0.992047
min,-3.021548,-4.199827,-3.122107,-3.872482
25%,-0.593294,-0.745365,-0.652834,-0.672029
50%,0.055366,-0.007627,-0.000041,-0.002602
75%,0.665971,0.632028,0.732216,0.647675
max,3.010387,3.504317,3.293896,3.023073


Suppose you wanted to find values in one of the columns exceeding 3 in absolute value:

In [36]:
col = data[2]
col[np.abs(col) > 3]

379   -3.122107
735    3.293896
951    3.100255
Name: 2, dtype: float64

To **select all rows having a value exceeding 3 or –3**, you can use the any method on a boolean DataFrame:

In [37]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
114,-0.465625,-1.783251,-0.780508,-3.145545
130,1.718287,0.751347,0.010592,3.023073
147,-3.021548,-0.660928,1.192849,-1.639071
238,0.241586,3.178667,1.378692,0.531702
379,-0.816156,1.133160,-3.122107,-0.828185
430,3.010387,0.050986,-1.334040,0.651489
503,-0.105821,-4.199827,-1.147031,-1.109149
599,0.454156,-1.570480,0.642079,-3.872482
735,-0.513845,0.042715,3.293896,-2.137431
866,-0.891818,-0.574611,0.435336,-3.108076


Values can be set based on these criteria. 
* Here is code to cap values outside the interval –3 to 3:
* The statement `np.sign(data)` produces 1 and –1 values based on whether the values in data are positive or negative:

In [38]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,1.0,-1.0
1,-1.0,-1.0,1.0,1.0
2,1.0,1.0,-1.0,1.0
3,1.0,1.0,1.0,1.0
4,1.0,1.0,-1.0,-1.0


In [39]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.042999,-0.035680,0.002094,-0.007903
std,0.934980,1.029497,0.988247,0.988169
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.593294,-0.745365,-0.652834,-0.672029
50%,0.055366,-0.007627,-0.000041,-0.002602
75%,0.665971,0.632028,0.732216,0.647675
max,3.000000,3.000000,3.000000,3.000000


### Permutation and Random Sampling
* Permuting (randomly reordering) a Series or the rows in a DataFrame is easy to do using the `numpy.random.permutation` function. 

In [40]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([0, 4, 3, 1, 2])

That array can then be used in **iloc-based indexing** or the equivalent `take` function:

In [41]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [42]:
df.take(sampler)

,0,1,2,3
0,0,1,2,3
4,16,17,18,19
3,12,13,14,15
1,4,5,6,7
2,8,9,10,11


To **select a random subset without replacement**, you can use the sample method on Series and DataFrame:

In [43]:
df.sample(n=3)

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
3,12,13,14,15


To generate a sample with replacement (to allow repeat choices), pass `replace=True` to sample:

In [44]:
df.sample(n=5,replace= True)

,0,1,2,3
4,16,17,18,19
2,8,9,10,11
0,0,1,2,3
1,4,5,6,7
4,16,17,18,19


### Computing Indicator/Dummy Variables
* Another type of transformation for statistical modeling or machine learning applications is converting a categorical variable into a “dummy” or “indicator” matrix. 
*  pandas has a `get_dummies` function

In [45]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [46]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In some cases, you may want to add a prefix to the columns in the indicator DataFrame

In [47]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


 Let’s look at the MovieLens 1M dataset

In [48]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('../datasets/movielens/movies.dat', sep='::',header=None, names=mnames,engine='python')
movies.head(10)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


Adding indicator variables for each genre requires a little bit of wrangling.

In [49]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

One way to construct the indicator DataFrame is to **start with a DataFrame of all zeros**:

In [50]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies.head()

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now, iterate through each movie and set entries in each row of dummies to 1. To do this, we use the `dummies.columns` to compute the column indices for each genre

In [51]:
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [52]:
 dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

Then, we can use `.iloc` to set values based on these indices:

In [53]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [54]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.head()

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,...,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


A useful recipe for statistical applications is to combine `get_dummies` with a discretization function like `cut`:

In [55]:
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [56]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0
